In [ ]:
! pip install transformers datasets
! pip install datasets

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import torch

from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from datasets import load_dataset, Dataset
from transformers import DataCollatorWithPadding
from transformers import create_optimizer
import tensorflow as tf

In [ ]:
# Load in dataset

In [ ]:
# Process data so that it can be appropriately used, meaning we will use the
# BERT text embedding
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
train_ds = Dataset.from_pandas(train[:int(np.ceil(0.8 * len(train)))])
val_ds = Dataset.from_pandas(train[int(np.ceil(0.8 * len(train))):])
test_ds = Dataset.from_pandas(test)

def tokens(ds):
    return tokenizer(ds["text"], truncation=True)
tokenized_train = train_ds.map(tokens, batched=True)
tokenized_val = val_ds.map(tokens, batched=True)
tokenized_test = test_ds.map(tokens, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
# Define training hyperparameters
init_lr=1e-5
batch_size = 2
num_epochs = 3
batches_per_epoch = len(tokenized_train) // batch_size
train_steps = int(batches_per_epoch * num_epochs)
# train_steps = int(len(tokenized_train)/batch_size * num_epochs)
optimizer, scheduler = create_optimizer(init_lr=init_lr, num_warmup_steps=0, num_train_steps=train_steps)


# Instantiate our model for text classification
model = TFAutoModelForSequenceClassification.from_pretrained(
"bert-base-uncased", num_labels=28
)

# Prepare the training and validation data and pass our Adam optimizer to the
# BERT model
train_set = model.prepare_tf_dataset(
    tokenized_train,
    batch_size=batch_size,
    collate_fn=data_collator,
)
val_set = model.prepare_tf_dataset(
    tokenized_val,
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)
test_set = model.prepare_tf_dataset(
    tokenized_test["text"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)
model.compile(optimizer=optimizer)

In [ ]:
# Train the model
model.fit(x=train_set, epochs = num_epochs)

In [ ]:
# Obtain validation predictions
preds = model.predict(val_set)["logits"]
class_preds = np.argmax(preds, axis=1)

In [ ]:
accuracy_score(yTv,class_preds)

In [ ]:
# Obtain prediction logits
final_preds = model.predict(test_set)["logits"]

In [ ]:
# Obtain predictions
final_class_preds = np.argmax(final_preds, axis=1)

In [ ]:
# Validation score performance
val_score = accuracy_score(yTv,class_preds)